# Setup 📔
This notebook will explain how to setup all required depedencies to run Vectrix. For some components you can use paid or hosted services, but for the sake of this setup we will focuse on open source solutions.

## Weaviate Vector Database
This is needed to store the embeddings and run the vector search. It's important to use a multi-modal embeddings engine since extracted documents can also contain images and graphs you might want to search on.

**The Docker Compose File**
```yaml
---
services:
  weaviate:
    command:
    - --host
    - 0.0.0.0
    - --port
    - '8080'
    - --scheme
    - http
    image: cr.weaviate.io/semitechnologies/weaviate:1.26.0
    ports:
    - 8080:8080
    - 50051:50051
    volumes:
    - ~/weaviate_data:/var/lib/weaviate
    restart: on-failure:0
    environment:
      SPELLCHECK_INFERENCE_API: 'http://text-spellcheck:8080'
      QUERY_DEFAULTS_LIMIT: 25
      AUTHENTICATION_ANONYMOUS_ACCESS_ENABLED: 'true'
      PERSISTENCE_DATA_PATH: '/var/lib/weaviate'
      DEFAULT_VECTORIZER_MODULE: 'text2vec-ollama'
      ENABLE_MODULES: 'text2vec-ollama,text-spellcheck,generative-ollama'
      CLUSTER_HOSTNAME: 'node1'
  text-spellcheck:
    image: cr.weaviate.io/semitechnologies/text-spellcheck-model:pyspellchecker-en
volumes:
  weaviate_data:
...
```

Note that we did not include an Ollama service since we will use the locally installed version of Ollama for this setup.

In [ ]:
# First download an embeddings model using ollama
!ollama pull mxbai-embed-large:335m

# Download the docker-compose file for Weaviate
!docker compose --project-name weaviate up -d

In [ ]:
poetry export -f requirements.txt --output requirements.txt --without-hashes

## Installing the PostgreSQL Database

In [ ]:
# Pull the postgres image, install the pgvector extension and run the container
!docker pull ankane/pgvector
!docker run -d --name paginx -e POSTGRES_PASSWORD=mysecretpassword -p 5432:5432 -e PG_EXTENSIONS="pgvector" ankane/pgvector